In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        if row['inc_angle'] == 'na':
            ang = -1
        else:
            ang = float(row['inc_angle'])
        
        band_3 = (band_1 - band_2)*ang/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.models import Model
from keras.applications.xception import Xception
def create_model():
    base_model = Xception(include_top=False, weights=None, input_tensor=None, input_shape=(75,75,3), pooling='max')
    x = Dense(512,activation='relu')(base_model.output)
    x = Dropout(0.2)(x)
    x = Dense(1,activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=x)
print('model model')


tmp_m = create_model()
tmp_m.summary()

model model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 37, 37, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 37, 37, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 37, 37, 32)   0           block1_conv1_bn[0][0]            
_________________________________________________________________________________________________

In [6]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator
import gc

def lr_f(epoch):
    if epoch<10:
        return 0.0008
    elif epoch<30:
        return 0.0004
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=99):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 12
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=80, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        
        del model
        gc.collect()
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

Epoch 1/80
Epoch 00001: val_loss improved from inf to 0.69331, saving model to best_m.h5
 - 27s - loss: 0.7634 - acc: 0.6471 - val_loss: 0.6933 - val_acc: 0.4505
Epoch 2/80
Epoch 00002: val_loss improved from 0.69331 to 0.69144, saving model to best_m.h5
 - 15s - loss: 0.5875 - acc: 0.7679 - val_loss: 0.6914 - val_acc: 0.5495
Epoch 3/80
Epoch 00003: val_loss did not improve
 - 15s - loss: 0.6179 - acc: 0.7116 - val_loss: 0.6975 - val_acc: 0.4505
Epoch 4/80
Epoch 00004: val_loss did not improve
 - 14s - loss: 0.4709 - acc: 0.7762 - val_loss: 0.6965 - val_acc: 0.4710
Epoch 5/80
Epoch 00005: val_loss did not improve
 - 14s - loss: 0.4674 - acc: 0.8063 - val_loss: 1.1122 - val_acc: 0.4579
Epoch 6/80
Epoch 00006: val_loss did not improve
 - 14s - loss: 0.4331 - acc: 0.8044 - val_loss: 0.9528 - val_acc: 0.5589
Epoch 7/80
Epoch 00007: val_loss improved from 0.69144 to 0.50724, saving model to best_m.h5
 - 15s - loss: 0.3748 - acc: 0.8595 - val_loss: 0.5072 - val_acc: 0.6748
Epoch 8/80
Epoch 0

Epoch 62/80
Epoch 00062: val_loss did not improve
 - 14s - loss: 0.2007 - acc: 0.9204 - val_loss: 0.2040 - val_acc: 0.9178
Epoch 63/80
Epoch 00063: val_loss did not improve
 - 14s - loss: 0.2244 - acc: 0.8972 - val_loss: 0.3060 - val_acc: 0.9103
Epoch 64/80
Epoch 00064: val_loss did not improve
 - 14s - loss: 0.1995 - acc: 0.9157 - val_loss: 0.3831 - val_acc: 0.8953
Epoch 65/80
Epoch 00065: val_loss did not improve
 - 14s - loss: 0.1849 - acc: 0.9270 - val_loss: 0.2308 - val_acc: 0.9121
Epoch 66/80
Epoch 00066: val_loss improved from 0.19202 to 0.18936, saving model to best_m.h5
 - 15s - loss: 0.1976 - acc: 0.9075 - val_loss: 0.1894 - val_acc: 0.9290
Epoch 67/80
Epoch 00067: val_loss did not improve
 - 14s - loss: 0.2163 - acc: 0.9101 - val_loss: 0.2026 - val_acc: 0.9159
Epoch 68/80
Epoch 00068: val_loss did not improve
 - 14s - loss: 0.1700 - acc: 0.9232 - val_loss: 0.2045 - val_acc: 0.9196
Epoch 69/80
Epoch 00069: val_loss did not improve
 - 14s - loss: 0.1838 - acc: 0.9401 - val_los

Epoch 43/80
Epoch 00043: val_loss improved from 0.16273 to 0.15914, saving model to best_m.h5
 - 15s - loss: 0.1868 - acc: 0.9167 - val_loss: 0.1591 - val_acc: 0.9402
Epoch 44/80
Epoch 00044: val_loss improved from 0.15914 to 0.15776, saving model to best_m.h5
 - 15s - loss: 0.2255 - acc: 0.9055 - val_loss: 0.1578 - val_acc: 0.9439
Epoch 45/80
Epoch 00045: val_loss improved from 0.15776 to 0.15203, saving model to best_m.h5
 - 15s - loss: 0.1883 - acc: 0.9232 - val_loss: 0.1520 - val_acc: 0.9439
Epoch 46/80
Epoch 00046: val_loss did not improve
 - 14s - loss: 0.2029 - acc: 0.9195 - val_loss: 0.1567 - val_acc: 0.9327
Epoch 47/80
Epoch 00047: val_loss did not improve
 - 14s - loss: 0.2022 - acc: 0.9167 - val_loss: 0.1627 - val_acc: 0.9346
Epoch 48/80
Epoch 00048: val_loss did not improve
 - 14s - loss: 0.2032 - acc: 0.9279 - val_loss: 0.1570 - val_acc: 0.9402
Epoch 49/80
Epoch 00049: val_loss did not improve
 - 14s - loss: 0.1918 - acc: 0.9241 - val_loss: 0.1758 - val_acc: 0.9178
Epoch 5

Epoch 26/80
Epoch 00026: val_loss did not improve
 - 14s - loss: 0.3055 - acc: 0.8745 - val_loss: 0.3356 - val_acc: 0.8596
Epoch 27/80
Epoch 00027: val_loss did not improve
 - 14s - loss: 0.3674 - acc: 0.8399 - val_loss: 0.2643 - val_acc: 0.8914
Epoch 28/80
Epoch 00028: val_loss did not improve
 - 14s - loss: 0.2810 - acc: 0.8764 - val_loss: 0.3188 - val_acc: 0.8614
Epoch 29/80
Epoch 00029: val_loss did not improve
 - 14s - loss: 0.2917 - acc: 0.8708 - val_loss: 0.4708 - val_acc: 0.8127
Epoch 30/80
Epoch 00030: val_loss did not improve
 - 14s - loss: 0.2987 - acc: 0.8811 - val_loss: 0.4745 - val_acc: 0.8502
Epoch 31/80
Epoch 00031: val_loss improved from 0.25516 to 0.24808, saving model to best_m.h5
 - 15s - loss: 0.2832 - acc: 0.8745 - val_loss: 0.2481 - val_acc: 0.8876
Epoch 32/80
Epoch 00032: val_loss did not improve
 - 14s - loss: 0.2562 - acc: 0.8839 - val_loss: 0.2560 - val_acc: 0.8914
Epoch 33/80
Epoch 00033: val_loss did not improve
 - 14s - loss: 0.2796 - acc: 0.8914 - val_los

In [7]:
import pickle
with open('../2nd_features/xception_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../2nd_features/xception_aug1_sub.csv', index=False)

0.184460799265
         id  is_iceberg
0  5941774d    0.020408
1  4023181e    0.965778
2  b20200e4    0.358534
3  e7f018bb    0.997419
4  4371c8c3    0.074239
